In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
password = getpass.getpass()

········


### 1- Create an SQL query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features

In [37]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''
with pred1 as (
    select year(rental_date) as rental_year, month(rental_date) as rental_month, inventory_id
    from rental
),
pred2 as (
    select p.rental_year, p.rental_month, i.film_id
    from pred1 p
    join inventory i
    on p.inventory_id = i.inventory_id
),
pred3 as (
    select rental_year, rental_month, film_id, count(film_id) as rented
    from pred2
    group by rental_year, rental_month, film_id
),
pred4 as(
    select p.rental_year, p.rental_month, f.film_id, p.rented, f.title, f.release_year, f.rental_rate, f.rental_duration, 
        f.length, f.rating, f.special_features, fc.category_id
    from film f
    left join pred3 p
    on p.film_id = f.film_id
    join film_category fc
    on f.film_id = fc.film_id
),
pred5 as(
    select p.rental_year, p.rental_month, p.film_id, p.rented, p.title, p.release_year, p.rental_rate, p.rental_duration, 
        p.length, p.rating, p.special_features, c.name as category
    from pred4 p
    join category c
    on p.category_id = c.category_id
)
select * from pred5;
        '''

### 2- Read the data into a Pandas dataframe

In [38]:
df = pd.read_sql_query(query, engine)
df.head()

,rental_year,rental_month,film_id,rented,title,release_year,rental_rate,rental_duration,length,rating,special_features,category
0,2005.0,8.0,19,7.0,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
1,2005.0,6.0,19,4.0,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
2,2005.0,7.0,19,8.0,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
3,2005.0,5.0,19,1.0,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
4,2006.0,2.0,19,1.0,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action


### 3- Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables

In [39]:
df.shape

(3712, 12)

In [41]:
df.isna().sum()

rental_year         42
rental_month        42
film_id              0
rented              42
title                0
release_year         0
rental_rate          0
rental_duration      0
length               0
rating               0
special_features     0
category             0
dtype: int64

In [43]:
df = df.fillna(0)

In [46]:
df['rental_year'] = df['rental_year'].astype(int)
df['rental_month'] = df['rental_month'].astype(int)
df['rented'] = df['rented'].astype(int)

In [50]:
df['rented'].value_counts()

1     724
2     619
3     502
4     356
5     297
6     283
8     220
7     207
9     167
10    129
11     84
12     46
0      42
13     17
14     16
15      2
16      1
Name: rented, dtype: int64

In [58]:
df2 = df.drop(df.loc[df['rental_year']== 2006].index)

,rental_year,rental_month,film_id,rented,title,release_year,rental_rate,rental_duration,length,rating,special_features,category
0,2005,8,19,7,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
1,2005,6,19,4,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
2,2005,7,19,8,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
3,2005,5,19,1,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
5,2005,5,21,2,AMERICAN CIRCUS,2006,4.99,3,129,R,"Commentaries,Behind the Scenes",Action
...,...,...,...,...,...,...,...,...,...,...,...,...
3707,2005,6,988,2,WORKER TARZAN,2006,2.99,7,139,R,"Trailers,Commentaries,Behind the Scenes",Travel
3708,2005,6,989,5,WORKING MICROCOSMOS,2006,4.99,4,74,R,"Commentaries,Deleted Scenes",Travel
3709,2005,7,989,10,WORKING MICROCOSMOS,2006,4.99,4,74,R,"Commentaries,Deleted Scenes",Travel
3710,2005,8,989,8,WORKING MICROCOSMOS,2006,4.99,4,74,R,"Commentaries,Deleted Scenes",Travel


### 4- Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable

In [53]:
df['last_month'] = []
for i in df['rental_month']

,rental_year,rental_month,film_id,rented,title,release_year,rental_rate,rental_duration,length,rating,special_features,category
0,2005,8,19,7,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
1,2005,6,19,4,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
2,2005,7,19,8,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
3,2005,5,19,1,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
4,2006,2,19,1,AMADEUS HOLY,2006,0.99,6,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action
...,...,...,...,...,...,...,...,...,...,...,...,...
3707,2005,6,988,2,WORKER TARZAN,2006,2.99,7,139,R,"Trailers,Commentaries,Behind the Scenes",Travel
3708,2005,6,989,5,WORKING MICROCOSMOS,2006,4.99,4,74,R,"Commentaries,Deleted Scenes",Travel
3709,2005,7,989,10,WORKING MICROCOSMOS,2006,4.99,4,74,R,"Commentaries,Deleted Scenes",Travel
3710,2005,8,989,8,WORKING MICROCOSMOS,2006,4.99,4,74,R,"Commentaries,Deleted Scenes",Travel


In [ ]:
# not sure what to do in this step...

### 5- Create a logistic regression model to predict this variable from the cleaned data

### 6- Evaluate the results (calculate the score of the model)